In [ ]:
import ujson as json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.plotly as py
import operator
import json, time
from __future__ import division
from moztelemetry import get_pings, get_pings_properties, get_one_ping_per_client
def fmt_date(d):
    return d.strftime("%Y%m%d")
def repartition(pipeline):
    return pipeline.repartition(MaxPartitions).cache()

%pylab inline

MaxPartitions = sc.defaultParallelism * 4
StartTime = datetime.datetime.now()

# Change this to False for local use.
RUN_AS_TELEMETRY_JOB = False

# Configuration for general data that spans all Firefox versions.
GeneralTimeWindow = 14
GeneralPingFraction = 0.40
SanityTestPingFraction = 0.5

In [ ]:
###########################################################
# Helper function block for fetching and filtering pings. #
###########################################################

def union_pipelines(a, b):
    if a is None:
        return b
    return a + b

def FetchRawPings(timeWindow, fraction, **kwargs):
    channel = kwargs.pop('channel', None)
    availDate = kwargs.pop('availDate', None)
    
    t1 = datetime.datetime.now() - datetime.timedelta(timeWindow)
    if availDate is not None:
        if availDate > t1:
            t1 = availDate
            availDays = (datetime.datetime.now() - t1).days
            assert availDays >= 0

            # Scale the fraction up to get roughly the same amount of pings.
            # Derivation, f=fraction, z=pings per day, d=days, a=available days
            #
            #     x*z*a = f*z*d
            #     x*a = f*d
            #     x = (f*d)/a
            fraction = min((fraction * timeWindow) / availDays, 1)
            timeWindow = availDays
            
    t1 = fmt_date(t1)
    t2 = fmt_date(datetime.datetime.now() - datetime.timedelta(0, 60*60)) # go back 1 hour
    
    kwargs = {
        'app': 'Firefox',
        'build_id': (t1, t2),
        'fraction': fraction,
    }
    info = {
        'fraction': fraction,
        'timeWindow': timeWindow,
        'channel': channel,
        'timestamp': datetime.datetime.utcnow(),
    }
    if isinstance(channel, tuple) or isinstance(channel, list):
        pings = None
        for c in channel:
            kwargs['channel'] = c
            pings = union_pipelines(pings, get_pings(sc, **kwargs))
    else:
        pings = get_pings(sc, **kwargs)
    return pings, info

# Transform each ping to make it easier to work with in later stages.
def Validate(p):
    name = p.get("environment/system/os/name") or 'w'
    version = p.get("environment/system/os/version") or '0'
    if name == 'Linux':
        p['OSVersion'] = None
        p['OS'] = 'Linux'
        p['OSName'] = 'Linux'
    elif name == 'Windows_NT':
        spmaj = p.get("environment/system/os/servicePackMajor") or '0'
        p['OSVersion'] = version + '.' + str(spmaj)
        p['OS'] = 'Windows-' + version + '.' + str(spmaj)
        p['OSName'] = 'Windows'
    elif name == 'Darwin':
        p['OSVersion'] = version
        p['OS'] = 'Darwin-' + version
        p['OSName'] = 'Darwin'
    else:
        return p
    
    # Telemetry data isn't guaranteed to be well-formed so unfortunately
    # we have to do some validation on it. If we get to the end, we set
    # p['valid'] to True, and this gets filtered over later. In addition
    # we have a wrapper below to help fetch strings that may be null.
    if not p.get("environment/build/version", None):
        return p
    p['FxVersion'] = p["environment/build/version"].split('.')[0]
    
    # Verify that we have at least one adapter.
    try:
        adapter = p["environment/system/gfx/adapters"][0]
    except:
        return p
    if adapter is None or not hasattr(adapter, '__getitem__'):
        return p
    
    def T(obj, key):
        return obj.get(key, None) or 'Unknown'
    
    # We store the device ID as a vendor/device string, because the device ID
    # alone is not enough to determine whether the key is unique.
    #
    # We also merge 'Intel Open Source Technology Center' with the device ID
    # that should be reported, 0x8086, for simplicity.
    vendorID = T(adapter, 'vendorID')
    if vendorID == u'Intel Open Source Technology Center':
        p['vendorID'] = u'0x8086'
    else:
        p['vendorID'] = vendorID
    p['deviceID'] = u'{0}/{1}'.format(p['vendorID'], T(adapter, 'deviceID'))
    p['driverVersion'] = u'{0}/{1}'.format(p['vendorID'], T(adapter, 'driverVersion'))
    if adapter['driverVersion']:
        p['driverShortVersion'] = '{0}/{1}'.format(p['vendorID'], '.'.join(T(adapter, 'driverVersion').split('.')[0:3]))
    else:
        p['driverShortVersion'] = '{0}/Unknown'.format(p['vendorID'])
        
    p['userPrefs'] = None
    if p.get('environment/settings', None) is not None:
        if p['environment/settings'].get('userPrefs', None) is not None:
            p['userPrefs'] = p['environment/settings']['userPrefs']
        
    p['valid'] = True
    return p

def reduce_pings(pings, schema='v2'):
    if schema == 'v4':
        prefix = 'payload/'
        clientId = 'clientId'
    else:
        prefix = ''
        clientId = 'clientID'
    return get_pings_properties(pings, [
      clientId,
      "creationDate",
      "environment/settings",
      "environment/build/version",
      "environment/build/buildId",
      "environment/system/os/name",
      "environment/system/os/version",
      "environment/system/os/servicePackMajor",
      "environment/system/os/servicePackMinor",
      "environment/system/gfx/adapters",
      "environment/system/gfx/features",
      "environment/system/gfx/monitors",
      prefix + "histograms/DEVICE_RESET_REASON",
      prefix + "histograms/GRAPHICS_SANITY_TEST",
      prefix + "histograms/GRAPHICS_SANITY_TEST_REASON",
      prefix + "histograms/GRAPHICS_DRIVER_STARTUP_TEST",
      prefix + "info/revision",
    ])

def FormatPings(pings, schema='v2'):
    pings = reduce_pings(pings, schema)
    pings = get_one_ping_per_client(pings)
    pings = pings.map(Validate)
    filtered_pings = pings.filter(lambda p: p.get('valid', False) == True)
    return filtered_pings.cache()

def FetchAndFormat(timeWindow, fraction, **kwargs):
    raw_pings, info = FetchRawPings(timeWindow, fraction, **kwargs)
    return FormatPings(raw_pings, kwargs.pop('schema', 'v2')), info

In [ ]:
##################################################################
# Helper function block for massaging pings into aggregate data. #
##################################################################

# Take each key in |b| and add it to |a|, accumulating its value into
# |a| if it already exists.
def combiner(a, b):
    result = a
    for key in b:
        countA = a.get(key, 0)
        countB = b[key]
        result[key] = countA + countB
    return result

# Return an aggregation based on combiner.
def aggregation(data, fn):
    view = data.map(fn)
    return view.reduceByKey(combiner)

# Helper for reduceByKey.
def map_x_to_y(data, sourceKey, destKey):
    def extract(p):
        return (p[sourceKey], { p[destKey]: 1 })
    return aggregation(data, extract)

# Helper for reduceByKey => count.
def map_x_to_count(data, sourceKey):
    def extract(p):
        return (p[sourceKey],)
    return data.map(extract).countByKey()

# After reduceByKey(combiner), we get a mapping like:
#  key => { variable => value }
#
# This function collapses 'variable' instances below a threshold into
# a catch-all identifier ('Other').
def coalesce_to_n_items(agg, max_items):
    obj = []
    for superkey, breakdown in agg:
        if len(breakdown) <= max_items:
            obj += [(superkey, breakdown)]
            continue
        items = sorted(breakdown.items(), key=lambda obj: obj[1], reverse=True)
        new_breakdown = {k: v for k, v in items[0:max_items]}
        total = 0
        for k, v in items[max_items:]:
            total += v
        if total:
            new_breakdown['Other'] = new_breakdown.get('Other', 0) + total
        obj += [(superkey, new_breakdown)]
    return obj

In [ ]:
#############################
# Helper for writing files. #
#############################

def ApplyPingInfo(obj, **kwargs):
    if 'pings' not in kwargs:
        return
    pings, info = kwargs.pop('pings')
    obj['sessions'] = {
        'count': pings.count(),
        'days': info['timeWindow'],
        'timestamp': time.mktime(info['timestamp'].timetuple()),
        'fraction': info['fraction'],
        'channels': info['channel'],
    }
    
def Export(filename, obj):
    if RUN_AS_TELEMETRY_JOB:
        filename = os.path.join('output', filename)
    with open(filename, 'w') as fp:
        json.dump(obj, fp)
        
def TimedExport(filename, callback, **kwargs):
    start = datetime.datetime.now()
    
    obj = callback()
    ApplyPingInfo(obj, **kwargs)
    
    end = datetime.datetime.now()
    elapsed = end - start
    obj['phaseTime'] = elapsed.total_seconds()
    
    Export(filename, obj)
    export_time = datetime.datetime.now() - end
    
    print('Computed {0} in {1} seconds.'.format(filename, elapsed.total_seconds()))
    print('Exported {0} in {1} seconds.'.format(filename, export_time.total_seconds()))

In [ ]:
# Get a general ping sample across all Firefox channels.
GeneralPings, GeneralPingInfo = FetchAndFormat(GeneralTimeWindow, GeneralPingFraction)

# Windows gets some preferential breakdown treatment.
WindowsPings = GeneralPings.filter(lambda p: p['OSName'] == 'Windows')
WindowsPings = WindowsPings.cache()

In [ ]:
# Results by operating system.
def GetGeneralStatistics():
    # OSToVendor = map_x_to_y(GeneralPings, 'OSName', 'vendorID')
    # OSToDevice = map_x_to_y(GeneralPings, 'OSName', 'deviceID')
    OSShare = map_x_to_count(GeneralPings, 'OSName')

    # Results by Windows version.
    # WindowsToVendor = map_x_to_y(WindowsPings, 'OSVersion', 'vendorID')
    # WindowsToDevice = map_x_to_y(WindowsPings, 'OSVersion', 'deviceID')
    # WindowsToDriver = map_x_to_y(WindowsPings, 'OSVersion', 'driverVersion')
    WindowsShare = map_x_to_count(WindowsPings, 'OSVersion')
    DriverShare = map_x_to_count(WindowsPings, 'driverVersion')

    # Results by Firefox version.
    # FxToVendor = map_x_to_y(AllData, 'FxVersion', 'vendorID')
    # FxToDevice = map_x_to_y(AllData, 'FxVersion', 'deviceID')
    FxShare = map_x_to_count(GeneralPings, 'FxVersion')

    # Top-level stats.
    VendorShare = map_x_to_count(GeneralPings, 'vendorID')
    DeviceShare = map_x_to_count(GeneralPings, 'deviceID')
    
    return {
        'os': OSShare,
        'windows': WindowsShare,
        'firefox': FxShare,
        'vendors': VendorShare,
        'devices': DeviceShare,
    }
        
TimedExport(filename = 'general-statistics.json',
            callback = GetGeneralStatistics,
            pings = (GeneralPings, GeneralPingInfo))

In [ ]:
#############################
# Perform the TDR analysis. #
#############################
def GetTDRStatistics():
    NumTDRReasons = 8
    def ping_has_tdr_for(p, reason):
        return p['histograms/DEVICE_RESET_REASON'][reason] > 0

    # Specialized version of map_x_to_y, for TDRs. We cast to int because for
    # some reason the values Spark returns do not serialize with JSON.
    def map_reason_to_vendor(p, reason, destKey):
        return (int(reason), { p[destKey]: int(p['histograms/DEVICE_RESET_REASON'][reason]) })
    def map_vendor_to_reason(p, reason, destKey):
        return (p[destKey], { int(reason): int(p['histograms/DEVICE_RESET_REASON'][reason]) })

    # Filter out pings that do not have any TDR data. We expect this to be a huge reduction
    # in the sample set, and the resulting partition count gets way off. We repartition
    # immediately for performance.
    TDRSubset = WindowsPings.filter(lambda p: p.get('histograms/DEVICE_RESET_REASON', None) is not None)
    TDRSubset = TDRSubset.repartition(MaxPartitions)
    TDRSubset = TDRSubset.cache()

    # Aggregate the device reset data.
    TDRResults = TDRSubset.map(lambda p: p['histograms/DEVICE_RESET_REASON']).reduce(lambda x, y: x + y)

    # For each TDR reason, get a list tuple of (reason, vendor => resetCount). Then
    # we combine these into a single series.
    reason_to_vendor_tuples = None
    vendor_to_reason_tuples = None
    for reason in xrange(1, NumTDRReasons):
        subset = TDRSubset.filter(lambda p: ping_has_tdr_for(p, reason))
        subset = subset.cache()

        tuples = subset.map(lambda p: map_reason_to_vendor(p, reason, 'vendorID'))
        reason_to_vendor_tuples = union_pipelines(reason_to_vendor_tuples, tuples)

        tuples = subset.map(lambda p: map_vendor_to_reason(p, reason, 'vendorID'))
        vendor_to_reason_tuples = union_pipelines(vendor_to_reason_tuples, tuples)

    TDRReasonToVendor = reason_to_vendor_tuples.reduceByKey(combiner, MaxPartitions)
    TDRVendorToReason = vendor_to_reason_tuples.reduceByKey(combiner, MaxPartitions)
    
    return {
        'tdrPings': TDRSubset.count(),
        'results': [int(value) for value in TDRResults],
        'reasonToVendor': TDRReasonToVendor.collect(),
        'vendorToReason': TDRVendorToReason.collect(),
    }
    
# Write TDR statistics.
TimedExport(filename = 'tdr-statistics.json',
            callback = GetTDRStatistics,
            pings = (WindowsPings, GeneralPingInfo))

In [ ]:
# Get pings with graphics features. This landed in roughly the 7-19-2015 nightly.
FeaturesKey = 'environment/system/gfx/features'

WindowsFeaturePings, WindowsFeaturePingsInfo = FetchAndFormat(
    timeWindow = GeneralTimeWindow,
    fraction = SanityTestPingFraction,
    channel = 'nightly',
    availDate = datetime.datetime(2015, 7, 19, 18, 20, 59, 378130))

def windows_feature_filter(p):
    return p['OSName'] == 'Windows' and p.get(FeaturesKey) is not None

WindowsFeatures = WindowsFeaturePings.filter(windows_feature_filter)
WindowsFeatures = WindowsFeatures.cache()

In [ ]:
# Build graphics feature statistics.
def get_compositor(p):
    compositor = p[FeaturesKey].get('compositor', 'none')
    if compositor == 'none':
        userPrefs = p['userPrefs']
        if userPrefs is not None:
            omtc = userPrefs.get('layers.offmainthreadcomposition.enabled', True)
            if omtc != True:
                compositor = 'disabled'
    return (compositor,)

def get_d3d11_status(p):
    d3d11 = p[FeaturesKey].get('d3d11', None)
    if not hasattr(d3d11, '__getitem__'):
        return 'unknown'
    status = d3d11.get('status', 'unknown')
    if status != 'available':
        return status
    if d3d11.get('warp', False) == True:
        return 'warp'
    return d3d11.get('version', 'unknown')

def get_warp_status(p):
    if 'blacklisted' not in p[FeaturesKey]['d3d11']:
        return 'unknown'
    if p[FeaturesKey]['d3d11']['blacklisted'] == True:
        return 'blacklist'
    return 'device failure'

def get_d2d_status(p):
    d2d = p[FeaturesKey].get('d2d', None)
    if not hasattr(d2d, '__getitem__'):
        return ('unknown',)
    status = d2d.get('status', 'unknown')
    if status != 'available':
        return (status,)
    return (d2d.get('version', 'unknown'),)

def has_working_d3d11(p):
    d3d11 = p[FeaturesKey].get('d3d11', None)
    if d3d11 is None:
        return False
    return d3d11.get('status') == 'available'

def get_texture_sharing_status(p):
    return (p[FeaturesKey]['d3d11'].get('textureSharing', 'unknown'),)

# We skip certain windows versions in detail lists since this phase is
# very expensive to compute.
ImportantWindowsVersions = (
    '5.1.2',
    '5.1.3',
    '6.0.2',
    '6.1.0',
    '6.1.1',
    '6.2.0',
    '6.3.0',
    '10.0.0',
)

def GetWindowsFeatures():
    WindowsCompositorMap = WindowsFeatures.map(get_compositor).countByKey()
    D3D11StatusMap = WindowsFeatures.map(lambda p: (get_d3d11_status(p),)).countByKey()
    D2DStatusMap = WindowsFeatures.map(get_d2d_status).countByKey()
    
    warp_pings = WindowsFeatures.filter(lambda p: get_d3d11_status(p) == 'warp')
    warp_pings = repartition(warp_pings)
    WarpStatusMap = warp_pings.map(lambda p: (get_warp_status(p),)).countByKey()

    TextureSharingMap = WindowsFeatures.filter(has_working_d3d11).map(get_texture_sharing_status).countByKey()

    # Now, build the same data except per version.
    feature_pings_by_os = map_x_to_count(WindowsFeatures, 'OSVersion')
    WindowsFeaturesByVersion = {}
    for os_version in feature_pings_by_os:
        if os_version not in ImportantWindowsVersions:
            continue
        subset = WindowsFeatures.filter(lambda p: p['OSVersion'] == os_version)
        subset = repartition(subset)
        
        results = {
            'count': subset.count(),
            'compositors': subset.map(get_compositor).countByKey(),
        }
        try:
            if int(os_version.split('.')[0]) >= 6:
                results['d3d11'] = subset.map(lambda p: (get_d3d11_status(p),)).countByKey()
                results['d2d'] = subset.map(get_d2d_status).countByKey()
                
                warp_pings = subset.filter(lambda p: get_d3d11_status(p) == 'warp')
                results['warp'] = warp_pings.map(lambda p: (get_warp_status(p),)).countByKey()
        except:
            pass
        finally:
            # Free resources.
            warp_pings = None
            subset = None
        WindowsFeaturesByVersion[os_version] = results
    
    return {
        'all': {
            'compositors': WindowsCompositorMap,
            'd3d11': D3D11StatusMap,
            'd2d': D2DStatusMap,
            'textureSharing': TextureSharingMap,
            'warp': WarpStatusMap,
        },
        'byVersion': WindowsFeaturesByVersion,
    }

TimedExport(filename = 'windows-features.json',
            callback = GetWindowsFeatures,
            pings = (WindowsFeatures, WindowsFeaturePingsInfo))

In [ ]:
# Sanity test and crash analysis setup.
# The sanity test was only available in Firefox 41 and higher, so currently
# our analyses grabs a different sample of pings to be more accurate.
#
# Once the sanity test makes it to beta we can use the main data set.
sanity_test_ping_pool, sanity_test_info = FetchAndFormat(
    timeWindow = GeneralTimeWindow,
    fraction = SanityTestPingFraction,
    channel = ('nightly', 'aurora'))

# Set up constants.
SANITY_TEST_PASSED = 0
SANITY_TEST_FAILED_RENDER = 1
SANITY_TEST_FAILED_VIDEO = 2
SANITY_TEST_CRASHED = 3
SANITY_TEST_LAST_VALUE = 4
SANITY_TEST_REASON_FIRST_RUN = 0
SANITY_TEST_REASON_FIREFOX_CHANGED = 1
SANITY_TEST_REASON_DEVICE_CHANGED = 2
SANITY_TEST_REASON_DRIVRE_CHANGED = 3
SAINTY_TEST_REASON_LAST_VALUE = 4

SANITY_TEST = 'histograms/GRAPHICS_SANITY_TEST'
SANITY_TEST_REASON = 'histograms/GRAPHICS_SANITY_TEST_REASON'

In [ ]:
# Filter sanity test pings.
def ping_has_sanity_test(p):
    return p.get(SANITY_TEST, None) is not None

sanity_test_pings = sanity_test_ping_pool.filter(ping_has_sanity_test)
sanity_test_pings = repartition(sanity_test_pings)
sanity_test_windows_share = map_x_to_count(sanity_test_pings, 'OSVersion')

In [ ]:
#########################
# Sanity test analysis. #
#########################
def GetSanityTests():
    # Aggregate the sanity test data.
    SanityTestResults = sanity_test_pings.map(lambda p: p[SANITY_TEST]).reduce(lambda x, y: x + y)
    
    reason_pings = sanity_test_pings.filter(lambda p: p.get(SANITY_TEST_REASON, None) is not None)
    SanityTestReasons = reason_pings.map(lambda p: p[SANITY_TEST_REASON]).reduce(lambda x, y: x + y)

    sanity_test_by_vendor = None
    sanity_test_by_os = None
    sanity_test_by_device = None
    sanity_test_by_driver = None
    for value in xrange(SANITY_TEST_FAILED_RENDER, SANITY_TEST_LAST_VALUE):
        subset = sanity_test_pings.filter(lambda p: p[SANITY_TEST][value] > 0)
        subset = subset.cache()

        tuples = subset.map(lambda p: (value, { p['vendorID']: int(p[SANITY_TEST][value]) }))
        sanity_test_by_vendor = union_pipelines(sanity_test_by_vendor, tuples)
    
        tuples = subset.map(lambda p: (value, { p['OS']: int(p[SANITY_TEST][value]) }))
        sanity_test_by_os = union_pipelines(sanity_test_by_os, tuples)
    
        tuples = subset.map(lambda p: (value, { p['deviceID']: int(p[SANITY_TEST][value]) }))
        sanity_test_by_device = union_pipelines(sanity_test_by_device, tuples)
    
        tuples = subset.map(lambda p: (value, { p['driverShortVersion']: int(p[SANITY_TEST][value]) }))
        sanity_test_by_driver = union_pipelines(sanity_test_by_driver, tuples)
    
    SanityTestByVendor = sanity_test_by_vendor.reduceByKey(combiner)
    SanityTestByOS = sanity_test_by_os.reduceByKey(combiner)
    SanityTestByDevice = sanity_test_by_device.reduceByKey(combiner)
    SanityTestByDriver = sanity_test_by_driver.reduceByKey(combiner)
    
    return {
        'sanityTestPings': sanity_test_pings.count(),
        'results': [int(value) for value in SanityTestResults],
        'reasons': [int(value) for value in SanityTestReasons],
        'byVendor': SanityTestByVendor.collect(),
        'byOS': SanityTestByOS.collect(),
        'byDevice': coalesce_to_n_items(SanityTestByDevice.collect(), 10),
        'byDriver': coalesce_to_n_items(SanityTestByDriver.collect(), 10),
        'windows': sanity_test_windows_share,
    }
    
# Write Sanity Test statistics.
TimedExport(filename = 'sanity-test-statistics.json',
            callback = GetSanityTests,
            pings = (sanity_test_ping_pool, sanity_test_info))

In [ ]:
#################################
# Build sanity test crash data. #
#################################
def make_crash_report(p):
    obj = {
        'os': {
            'name': p['environment/system/os/name'],
            'version': p.get('environment/system/os/version', None),
            'servicePack': None,
        },
        'adapter': p['environment/system/gfx/adapters'][0],
        'build': {
            'version': p['environment/build/version'],
            'id': p.get('environment/build/buildId', None),
            'revision': p.get('info/revision', None),
        },
        'date': p.get('creationDate', None)
    }
    if obj['os']['name'] == 'Windows_NT':
        spmaj = p.get('environment/system/os/servicePackMajor', 0)
        spmin = p.get('environment/system/os/servicePackMinor', 0)        
        if spmaj:
            if spmin:
                obj['os']['servicePack'] = '{0}.{1}'.format(spmaj, spmin)
            else:
                obj['os']['servicePack'] = '{0}'.format(spmaj)
    return obj

In [ ]:
#############################
# Sanity test crash reports #
#############################
def GetSanityTestCrashes():
    def ping_has_sanity_test_crash(p):
        return p[SANITY_TEST][SANITY_TEST_CRASHED] > 0

    sanity_test_crash_pings = sanity_test_pings.filter(ping_has_sanity_test_crash)
    reports = sanity_test_crash_pings.map(make_crash_report)
    return {
        'sanityTestPings': sanity_test_pings.count(),
        'reports': reports.collect(),
    }
    
TimedExport(filename = 'sanity-test-crash-reports.json',
            callback = GetSanityTestCrashes,
            pings = (sanity_test_ping_pool, sanity_test_info))

In [ ]:
STARTUP_TEST_KEY = 'histograms/GRAPHICS_DRIVER_STARTUP_TEST'
STARTUP_OK = 0
STARTUP_ENV_CHANGED = 1
STARTUP_CRASHED = 2
STARTUP_ACCEL_DISABLED = 3
    
def GetStartupTests():
    startup_test_pings = sanity_test_ping_pool.filter(lambda p: p.get(STARTUP_TEST_KEY, None) is not None)
    startup_test_pings = startup_test_pings.repartition(MaxPartitions)
    startup_test_pings = startup_test_pings.cache()

    StartupTestResults = startup_test_pings.map(lambda p: p[STARTUP_TEST_KEY]).reduce(lambda x, y: x + y)

    startup_test_crashes = startup_test_pings.filter(lambda p: p[STARTUP_TEST_KEY][STARTUP_CRASHED] > 0)
    StartupTestCrashes = startup_test_crashes.map(make_crash_report)
    
    return {
        'startupTestPings': startup_test_pings.count(),
        'results': [int(i) for i in StartupTestResults],
        'reports': StartupTestCrashes.collect(),
        'windows': sanity_test_windows_share,
    }
    
# Write startup test results.
TimedExport(filename = 'startup-test-statistics.json',
            callback = GetStartupTests,
            pings = (sanity_test_ping_pool, sanity_test_info))

In [ ]:
MonitorsKey = 'environment/system/gfx/monitors'

def get_monitor_count(p):
    monitors = p.get(MonitorsKey, None)
    try:
        return len(monitors)
    except:
        return 0
    
def GetMonitorStatistics():
    def get_monitor_rdds_for_index(data, i):
        def get_refresh_rate(p):
            refreshRate = p[MonitorsKey][i].get('refreshRate', 0)
            return refreshRate if refreshRate > 1 else 'Unknown'
        def get_resolution(p):
            width = p[MonitorsKey][i].get('screenWidth', 0)
            height = p[MonitorsKey][i].get('screenHeight', 0)
            if width == 0 or height == 0:
                return 'Unknown'
            return '{0}x{1}'.format(width, height)
    
        monitors_at_index = data.filter(lambda p: get_monitor_count(p) == monitor_count)
        monitors_at_index = repartition(monitors_at_index)
        refresh_rates = monitors_at_index.map(lambda p: (get_refresh_rate(p),))
        resolutions = monitors_at_index.map(lambda p: (get_resolution(p),))
        return refresh_rates, resolutions
    
    MonitorCounts = sanity_test_ping_pool.map(lambda p: (get_monitor_count(p),)).countByKey()
    MonitorCounts.pop(0, None)

    refresh_rates = None
    resolutions = None
    for monitor_count in MonitorCounts:
        rate_subset, res_subset = get_monitor_rdds_for_index(sanity_test_ping_pool, monitor_count - 1)
        refresh_rates = union_pipelines(refresh_rates, rate_subset)
        resolutions = union_pipelines(resolutions, res_subset)
    
    MonitorRefreshRates = refresh_rates.countByKey()
    MonitorResolutions = resolutions.countByKey()
    
    return {
        'counts': MonitorCounts,
        'refreshRates': MonitorRefreshRates,
        'resolutions': MonitorResolutions,
    }

TimedExport(filename = 'monitor-statistics.json',
            callback = GetMonitorStatistics,
            pings = (sanity_test_ping_pool, sanity_test_info))

In [ ]:
EndTime = datetime.datetime.now()
TotalElapsed = (EndTime - StartTime).total_seconds()

print('Total time: {0}'.format(TotalElapsed))